In [1]:
import os
import shutil

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.utils.class_weight import compute_class_weight

from utils import augment_images, create_model_v2, flatten_datasets
from config import config

import pandas as pd

In [2]:
data_dir = "dataset/2-cropped-v3"

train_set, test_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    seed=config["seed_value"],
    image_size=config["img_shape"],
    batch_size=config["batch_size"],
    label_mode="categorical",
    subset="both",
)

train_images, train_labels = flatten_datasets(train_set)
test_images, test_labels = flatten_datasets(test_set)

Found 447 files belonging to 2 classes.
Using 358 files for training.
Using 89 files for validation.


In [3]:
flag = 2

models = {
    1: {
        "base_model": tf.keras.applications.MobileNetV3Small(
            weights="imagenet",
            input_shape=config["input_shape"],
            include_top=False,
            pooling="avg",
        ),
        "model_name": "mobilenetv3small",
    },
    2: {
        "base_model": tf.keras.applications.EfficientNetV2B0(
            weights="imagenet",
            input_shape=config["input_shape"],
            include_top=False,
            pooling="avg",
        ),
        "model_name": "efficientnetv2b0",
    },
}

base_model = models[flag]["base_model"]
model_name = models[flag]["model_name"]

In [4]:
dropout_histories = []
dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]

# Define the K-fold Cross Validator with 2 folds
cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=config["seed_value"])

for dropout_rate in dropout_rates:
    kfold_histories = []

    # Loop over the dataset to create separate folds
    for i, (train_idx, valid_idx) in enumerate(
        cv.split(train_images, np.argmax(train_labels, axis=1))
    ):
        print(
            f"Fold {i + 1}, Training {model_name} with dropout rate {dropout_rate}..."
        )

        # Create a new model instance
        model = create_model_v2(base_model, config, dropout_rate)
        X_train, y_train = train_images[train_idx], train_labels[train_idx]
        X_valid, y_valid = train_images[valid_idx], train_labels[valid_idx]

        # Augment training set
        X_train, y_train = augment_images(X_train, y_train, 5)

        # Compute weights
        weights = compute_class_weight(
            class_weight="balanced",
            classes=np.unique([0, 1]),
            y=y_train.argmax(axis=1),
        )
        weights = dict(zip(np.unique([0, 1]), weights))

        # Define checkpoint path and checkpoint callback
        path = f"checkpoints/dropout_tests/{model_name}/{dropout_rate}/fold_{i+1}"
        if os.path.exists(path):
            shutil.rmtree(path)
        os.makedirs(path)

        checkpoint_path = path + "/cp-{epoch:04d}.ckpt"

        # Save the weights using the `checkpoint_path` format
        model.save_weights(checkpoint_path.format(epoch=0))

        # Define callbacks
        callbacks = [
            tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_path,
                monitor="val_loss",
                save_best_only=True,
                save_weights_only=True,
                verbose=1,
            ),
            tf.keras.callbacks.EarlyStopping(
                monitor="val_loss", mode="min", patience=10
            ),
        ]

        # Fit the model on the train set and evaluate on the validation set
        history = model.fit(
            X_train,
            y_train,
            batch_size=config["batch_size"],
            epochs=config["epochs"],
            class_weight=weights,
            validation_data=(X_valid, y_valid),
            verbose=1,
            callbacks=callbacks,
        )

        # Append to kfold histories
        kfold_histories.append(history)

    # Append to dropout histories
    dropout_histories.append(kfold_histories)

Fold 1, Training efficientnetv2b0 with dropout rate 0.0...
Epoch 1/150
17/17 [==============================] - ETA: 0s - loss: 0.6406 - precision: 0.5447 - recall: 0.5447 - f1_score: 0.6445 - false_negatives: 489.0000 - true_positives: 585.0000 - false_positives: 489.0000 - true_negatives: 585.0000
Epoch 1: val_loss improved from inf to 0.57738, saving model to checkpoints/dropout_tests/efficientnetv2b0/0.0/fold_1/cp-0001.ckpt
17/17 [==============================] - 17s 285ms/step - loss: 0.6406 - precision: 0.5447 - recall: 0.5447 - f1_score: 0.6445 - false_negatives: 489.0000 - true_positives: 585.0000 - false_positives: 489.0000 - true_negatives: 585.0000 - val_loss: 0.5774 - val_precision: 0.7151 - val_recall: 0.7151 - val_f1_score: 0.7835 - val_false_negatives: 51.0000 - val_true_positives: 128.0000 - val_false_positives: 51.0000 - val_true_negatives: 128.0000
Epoch 2/150
17/17 [==============================] - ETA: 0s - loss: 0.5839 - precision: 0.6862 - recall: 0.6862 - f1_sc

: 

In [ ]:
dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]

# save history at each dropout rate to csv
for i, dropout_history in enumerate(dropout_histories):
    path = f"results/dropout_tests/{model_name}/{dropout_rates[i]}"
    if os.path.exists(path):
        shutil.rmtree(path, ignore_errors=True)
    os.makedirs(path)

    for j, kfold_history in enumerate(dropout_history):
        df = pd.DataFrame(kfold_history.history)

        df.to_csv(
            path + f"/fold_{j+1}.csv",
            index=False,
        )

In [ ]:
# plot kfold histories per dropout rate
dropout_rates = [0.2, 0.3, 0.4, 0.5]
ax, fig = plt.subplots(figsize=(8, 20), dpi=600)
for i, histories in enumerate(dropout_histories):
    plt.subplot(4, 1, i + 1)
    mean_val_loss = np.mean(
        [history.history["val_f1_score"] for history in histories], axis=0
    )
    std_val_loss = np.std(
        [history.history["val_f1_score"] for history in histories], axis=0
    )

    plt.plot(mean_val_loss, label="Validation Loss")
    plt.fill_between(
        range(len(mean_val_loss)),
        mean_val_loss - std_val_loss,
        mean_val_loss + std_val_loss,
        alpha=0.2,
    )

    plt.title(f"Validation Loss for Dropout Rate {dropout_rates[i]}")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")

    plt.xlim([0, config["epochs"] - 1])
    plt.ylim([0, 1])
    plt.legend(loc="lower right")

plt.tight_layout()
plt.show()

: 

: 